In [1]:
# %load /cluster/home/myurchikova/github/projects2020_ohsu/eth/learning_Master_thesis/TASKS/func/base_imports.py
import pandas as pd
import numpy as np
import tqdm 
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import tarfile
import re
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
import sys
sys.path.append(r"/cluster/home/myurchikova/github/projects2020_ohsu/eth/learning_Master_thesis/TASKS/func")
from config import *
from functions import *




In [2]:

##Generation ETH in dataframe format

OVARIAN_ETH='/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_intermediate/complete_cancer_candidates_order_r_complete.tsv.gz'


##Generation OHSU in dataframe format
#Breast/Ovarian
OVARIAN_OHSU="/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/OHSU_June2023_filter-debug_complete-annotated-shortlist.tsv.gz"
BATCH_PATH = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/batch_to_gene.txt'

samples = ['TCGA-25-1319-01A-01R-1565-13',
               'TCGA-25-1313-01A-01R-1565-13',
               'TCGA-61-2008-01A-02R-1568-13',
               'TCGA-24-1431-01A-01R-1566-13',
               'TCGA-24-2298-01A-01R-1569-13']
MAIN = [
        'kmer',
        'batch',
]
                                 
COLS_ETH = [
            'coord',
            'kmer',
            'batch',
            'TCGA25131901A01R156513all',
            'TCGA25131301A01R156513all',
            'TCGA61200801A02R156813all',
            'TCGA24143101A01R156613all',
            'TCGA24229801A01R156913all',
           ]
ETH_COLMNS= {
             'TCGA25131901A01R156513all':'TCGA25131901A01R156513',
             'TCGA25131301A01R156513all':'TCGA25131301A01R156513',
             'TCGA61200801A02R156813all':'TCGA61200801A02R156813',
             'TCGA24143101A01R156613all':'TCGA24143101A01R156613',
             'TCGA24229801A01R156913all':'TCGA24229801A01R156913',
            }
EXPRESSION=['TCGA25131901A01R156513',
            'TCGA25131301A01R156513',
            'TCGA61200801A02R156813',
            'TCGA24143101A01R156613',
            'TCGA24229801A01R156913',]

In [3]:
eth_df=pd.read_csv(OVARIAN_ETH, usecols = COLS_ETH, sep = '\t', low_memory=False)
batch_to_gene = pd.read_csv(BATCH_PATH, names = ['gene_id', 'batch'], low_memory=False)
eth_df.head()

,kmer,coord,batch,TCGA25131901A01R156513all,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all
0,RKSTQMPCT,92379857:92379859:92611313:92611338:None:None,56543,20.982616,8.422932,17.304198,16.069005,19.585264
1,RKSTQMPCT,92379857:92379859:92611313:92611338:None:None,56543,20.982616,8.422932,17.304198,16.069005,19.585264
2,RKSTQMPCT,92379857:92379859:92611313:92611338:None:None,56543,20.982616,8.422932,17.304198,16.069005,19.585264
3,RKSTQMPCT,92379857:92379859:92611313:92611338:None:None,56543,20.982616,8.422932,17.304198,16.069005,19.585264
4,RKSTQMPCT,92379857:92379859:92611313:92611338:None:None,56543,20.982616,8.422932,17.304198,16.069005,19.585264


In [4]:
ohsu_df=pd.read_csv(OVARIAN_OHSU, sep = '\t', low_memory=False)
ohsu_df.head()

,jx,TCGA-24-1431-01A-01R-1566-13,TCGA-24-2298-01A-01R-1569-13,TCGA-25-1313-01A-01R-1565-13,TCGA-25-1319-01A-01R-1565-13,TCGA-61-2008-01A-02R-1568-13,TCGA-A2-A0D2-01A-21R-A034-07,TCGA-A2-A0SX-01A-12R-A084-07,TCGA-AO-A0JM-01A-21R-A056-07,TCGA-BH-A18V-01A-11R-A12D-07,...,modified_upstream_txs,in-frame_all-transcript_biexons,in-frame_nonhanging-tx_biexons,in-frame_peptide_sequence,hanging_txs_included_inframe_pepseqs,prefiltered_in-frame_epitopes,prefiltered_inframe_epitope_count,in-frame_neoepitopes,in-frame_neoepitope_count,frame-agnostic_all-transcript_biexons
0,chr10;48726;48803;-,0.000000,2.176140,0.842293,2.517914,0.961344,0.000000,5.700962,2.102386,1.237161,...,['ENST00000561967.1.MOD.CHR10.48726.48803.MINU...,LTQIGQCGNQIGAKFWEVISDEHAIDSAGTYHGDSHLQLERINVYY...,LTQIGQCGNQIGAKFWEVISDEHAIDSAGTYHGDSHLQLERINVYY...,NVYYNEASGGRYVPRAV;STCTTTRPAVAGQCGA,NVYYNEASGGRYVPRAV;STCTTTRPAVAGQCGA,NVYYNEASG;VYYNEASGG;YYNEASGGR;YNEASGGRY;NEASGG...,17.0,TTRPAVAGQ;TRPAVAGQC;RPAVAGQCG;PAVAGQCGA,4.0,LTQIGQCGNQIGAKFWEVISDEHAIDSAGTYHGDSHLQLERINVYY...
1,chr10;277578;281199;-,2.295572,30.465966,16.003571,10.071656,26.917642,51.966700,53.589040,31.535793,50.723584,...,['ENST00000280886.12.MOD.CHR10.277578.281199.M...,RTELTDANGERHDALYVVGALDEAMELRGMRYHPIDIETSVIRAHK...,RTELTDANGERHDALYVVGALDEAMELRGMRYHPIDIETSVIRAHK...,RAHKSVTECAVFTWTNL,RAHKSVTECAVFTWTNL,RAHKSVTEC;AHKSVTECA;HKSVTECAV;KSVTECAVF;SVTECA...,9.0,NaN,0.0,RTELTDANGERHDALYVVGALDEAMELRGMRYHPIDIETSVIRAHK...
2,chr10;280261;281199;-,0.000000,0.000000,0.000000,0.000000,0.961344,0.000000,0.000000,6.307159,2.474321,...,['ENST00000280886.12.MOD.CHR10.280261.281199.M...,RTELTDANGERHDALYVVGALDEAMELRGMRYHPIDIETSVIRAHK...,RTELTDANGERHDALYVVGALDEAMELRGMRYHPIDIETSVIRAHK...,RAHKSVTEC,RAHKSVTEC,RAHKSVTEC,1.0,NaN,0.0,RTELTDANGERHDALYVVGALDEAMELRGMRYHPIDIETSVIRAHK...
3,chr10;281324;283271;-,3.443358,23.212165,18.530451,15.107483,20.188231,32.821074,33.065578,44.150110,40.826300,...,['ENST00000280886.12.MOD.CHR10.281324.283271.M...,IWVHSAHNASGYFTIYGDESLQSDHFNSRLSFGDTQTIWARTGYLG...,IWVHSAHNASGYFTIYGDESLQSDHFNSRLSFGDTQTIWARTGYLG...,TELTDANGERHDALYVV,TELTDANGERHDALYVV,TELTDANGE;ELTDANGER;LTDANGERH;TDANGERHD;DANGER...,9.0,NaN,0.0,IWVHSAHNASGYFTIYGDESLQSDHFNSRLSFGDTQTIWARTGYLG...
4,chr10;283447;286272;-,1.147786,41.346668,19.372744,6.714437,28.840330,61.539513,62.710579,37.842952,92.787044,...,['ENST00000280886.12.MOD.CHR10.283447.286272.M...,ALRHDRVRLVERGSPHSLPLMESGKILPGVRIIIANPETKGPLGDS...,ALRHDRVRLVERGSPHSLPLMESGKILPGVRIIIANPETKGPLGDS...,PLGDSHLGEIWVHSAH,PLGDSHLGEIWVHSAH,PLGDSHLGE;LGDSHLGEI;GDSHLGEIW;DSHLGEIWV;SHLGEI...,8.0,NaN,0.0,ALRHDRVRLVERGSPHSLPLMESGKILPGVRIIIANPETKGPLGDS...


In [5]:
def get_junction_coordinates(df, coordinates_col, sep=':'):
    df['strand'] = None
    df['junction_coordinate'] = None

    for idx, row in tqdm.tqdm(df.iterrows()):
        kmer_coordinates = [int(x) for x in row[coordinates_col].split(sep) if x !='None']

        if kmer_coordinates[1] < kmer_coordinates[2]: # order strand +

            df.loc[idx, 'strand'] = '+'
            if len(kmer_coordinates) == 4:  # 2 exons
                df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in kmer_coordinates[1:3]])
            elif len(kmer_coordinates) == 6:
                df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in kmer_coordinates[1:5]])
        else: # order strand +
            df.loc[idx, 'strand'] = '-'
            if len(kmer_coordinates) == 4:  # 2 exons
                df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in [kmer_coordinates[3],
                                                                                kmer_coordinates[0]]])
            elif len(kmer_coordinates) == 6:
                df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in [kmer_coordinates[3],
                                                                                kmer_coordinates[0],
                                                                                kmer_coordinates[2],
                                                                                kmer_coordinates[5]
                                                                               ]])
    return df

In [6]:
eth_df=get_junction_coordinates(eth_df,'coord')
ETH_df = pd.merge(batch_to_gene,eth_df, on=['batch'])
ETH_df.drop(columns=['batch'],inplace=True)
ohsu_df.rename(columns = ETH_COLMNS, inplace = True)
ETH_df.rename(columns = ETH_COLMNS, inplace = True)
ETH_df

10856541it [1:40:54, 1842.23it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

13082964it [2:00:44, 2024.83it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

46132795it [5:24:37, 2368.49it/s]


,gene_id,kmer,coord,TCGA25131901A01R156513,TCGA25131301A01R156513,TCGA61200801A02R156813,TCGA24143101A01R156613,TCGA24229801A01R156913,strand,junction_coordinate
0,ENSG00000169962.5,NQTTSPAPF,1333118:1333124:1333505:1333526:None:None,0.000000,0.0,0.0,4.591144,0.72538,+,1333124:1333505
1,ENSG00000169962.5,NQTTSPAPF,1333118:1333124:1333505:1333526:None:None,0.000000,0.0,0.0,4.591144,0.72538,+,1333124:1333505
2,ENSG00000169962.5,DNQTTSPAP,1333115:1333124:1333505:1333523:None:None,0.000000,0.0,0.0,4.591144,0.72538,+,1333124:1333505
3,ENSG00000169962.5,DNQTTSPAP,1333115:1333124:1333505:1333523:None:None,0.000000,0.0,0.0,4.591144,0.72538,+,1333124:1333505
4,ENSG00000169962.5,QTTSPAPFV,1333121:1333124:1333505:1333529:None:None,0.000000,0.0,0.0,4.591144,0.72538,+,1333124:1333505
...,...,...,...,...,...,...,...,...,...,...
46118846,ENSG00000171533.11,KSCTSVEFP,75608108:75608111:75607238:75607262:None:None,4.196523,0.0,0.0,0.000000,0.00000,-,75607262:75608108
46118847,ENSG00000171533.11,KSCTSVEFP,75608108:75608111:75607238:75607262:None:None,4.196523,0.0,0.0,0.000000,0.00000,-,75607262:75608108
46118848,ENSG00000171533.11,KSCTSVEFP,75608108:75608111:75607238:75607262:None:None,4.196523,0.0,0.0,0.000000,0.00000,-,75607262:75608108
46118849,ENSG00000171533.11,KSCTSVEFP,75608108:75608111:75607238:75607262:None:None,4.196523,0.0,0.0,0.000000,0.00000,-,75607262:75608108


In [7]:
def change_column_names(ohsu_df):
    sample_col = [col for col in ohsu_df if col.startswith('TCGA')]
    new_col = [s.replace('-', '') for s in sample_col]
    #new_col = [s.replace('-', '') + 'all' for s in sample_col]
    d = dict(zip(sample_col, new_col))
    ohsu_df.rename(columns = d, inplace = True)
    return ohsu_df


def preprocess_ohsu(df):
    pd.options.mode.chained_assignment = None #None - ignoring the warning, This line disables a warning that occurs when assigning values to a DataFrame. It prevents the warning from being displayed.
    df = df[df['gene_id'].notna()] #notna() - detect existing (non-missing) values. This line filters out rows where the 'gene_id' column is not null. It removes any rows with missing gene IDs.
    df = df[df['kmer'].notna()] #This line filters out rows where the 'kmer' column is not null. It removes any rows with missing kmer values.
    df.loc[:, 'kmer'] = df['kmer'].apply(lambda s: s.split(';')) #This line splits the values in the 'kmer' column by the ';' delimiter and assigns the resulting list to the 'kmer' column. It splits a string into multiple values.
    df = df.explode('kmer') #This line explodes the 'kmer' column, which means it creates multiple rows for each value in the 'kmer' column. It expands a list into separate rows.
    df.loc[:, 'gene_id'] = df['gene_id'].apply(lambda s: s.split(','))
    df = df.explode('gene_id')

    return df

In [8]:
def ohsu_to_eth_coord(df, col = 'jx', new_col = 'jx_shifted', sep = ';'):
    tmp_jx = df[col].str.split(sep,  expand = True)
    df[new_col] = tmp_jx[0] + sep + (tmp_jx[1].astype(int) - 1).astype(str) + sep + tmp_jx[2] + sep + tmp_jx[3]
    return df

In [9]:
ohsu_df.rename(columns = {'in-frame_neoepitopes': 'kmer'}, inplace = True)
ohsu_df = ohsu_to_eth_coord(ohsu_df)
ohsu_df = change_column_names(ohsu_df)
ohsu_df = preprocess_ohsu(ohsu_df)
ohsu_df['junction_coordinate'] = ohsu_df['jx_shifted'].apply(lambda x: ':'.join(x.split(';')[1:3]))
print(ohsu_df['junction_coordinate'])

0                  48725:48803
0                  48725:48803
0                  48725:48803
0                  48725:48803
29               390095:390274
                  ...         
1344358    109916656:109919488
1344358    109916656:109919488
1344358    109916656:109919488
1344358    109916656:109919488
1344358    109916656:109919488
Name: junction_coordinate, Length: 3182186, dtype: object


In [10]:
ohsu_df.head()


,jx,TCGA24143101A01R156613,TCGA24229801A01R156913,TCGA25131301A01R156513,TCGA25131901A01R156513,TCGA61200801A02R156813,TCGAA2A0D201A21RA03407,TCGAA2A0SX01A12RA08407,TCGAAOA0JM01A21RA05607,TCGABHA18V01A11RA12D07,...,in-frame_nonhanging-tx_biexons,in-frame_peptide_sequence,hanging_txs_included_inframe_pepseqs,prefiltered_in-frame_epitopes,prefiltered_inframe_epitope_count,kmer,in-frame_neoepitope_count,frame-agnostic_all-transcript_biexons,jx_shifted,junction_coordinate
0,chr10;48726;48803;-,0.0,2.17614,0.842293,2.517914,0.961344,0.0,5.700962,2.102386,1.237161,...,LTQIGQCGNQIGAKFWEVISDEHAIDSAGTYHGDSHLQLERINVYY...,NVYYNEASGGRYVPRAV;STCTTTRPAVAGQCGA,NVYYNEASGGRYVPRAV;STCTTTRPAVAGQCGA,NVYYNEASG;VYYNEASGG;YYNEASGGR;YNEASGGRY;NEASGG...,17.0,TTRPAVAGQ,4.0,LTQIGQCGNQIGAKFWEVISDEHAIDSAGTYHGDSHLQLERINVYY...,chr10;48725;48803;-,48725:48803
0,chr10;48726;48803;-,0.0,2.17614,0.842293,2.517914,0.961344,0.0,5.700962,2.102386,1.237161,...,LTQIGQCGNQIGAKFWEVISDEHAIDSAGTYHGDSHLQLERINVYY...,NVYYNEASGGRYVPRAV;STCTTTRPAVAGQCGA,NVYYNEASGGRYVPRAV;STCTTTRPAVAGQCGA,NVYYNEASG;VYYNEASGG;YYNEASGGR;YNEASGGRY;NEASGG...,17.0,TRPAVAGQC,4.0,LTQIGQCGNQIGAKFWEVISDEHAIDSAGTYHGDSHLQLERINVYY...,chr10;48725;48803;-,48725:48803
0,chr10;48726;48803;-,0.0,2.17614,0.842293,2.517914,0.961344,0.0,5.700962,2.102386,1.237161,...,LTQIGQCGNQIGAKFWEVISDEHAIDSAGTYHGDSHLQLERINVYY...,NVYYNEASGGRYVPRAV;STCTTTRPAVAGQCGA,NVYYNEASGGRYVPRAV;STCTTTRPAVAGQCGA,NVYYNEASG;VYYNEASGG;YYNEASGGR;YNEASGGRY;NEASGG...,17.0,RPAVAGQCG,4.0,LTQIGQCGNQIGAKFWEVISDEHAIDSAGTYHGDSHLQLERINVYY...,chr10;48725;48803;-,48725:48803
0,chr10;48726;48803;-,0.0,2.17614,0.842293,2.517914,0.961344,0.0,5.700962,2.102386,1.237161,...,LTQIGQCGNQIGAKFWEVISDEHAIDSAGTYHGDSHLQLERINVYY...,NVYYNEASGGRYVPRAV;STCTTTRPAVAGQCGA,NVYYNEASGGRYVPRAV;STCTTTRPAVAGQCGA,NVYYNEASG;VYYNEASGG;YYNEASGGR;YNEASGGRY;NEASGG...,17.0,PAVAGQCGA,4.0,LTQIGQCGNQIGAKFWEVISDEHAIDSAGTYHGDSHLQLERINVYY...,chr10;48725;48803;-,48725:48803
29,chr10;390096;390274;-,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,2.102386,0.000000,...,CHKGLPKSPTGEIPQFKGWPKLLWFVTESKHLSKPPRDWFPHIKDA...,IKDANNDTEYKTCKDGS,IKDANNDTEYKTCKDGS,IKDANNDTE;KDANNDTEY;DANNDTEYK;ANNDTEYKT;NNDTEY...,9.0,IKDANNDTE,8.0,CHKGLPKSPTGEIPQFKGWPKLLWFVTESKHLSKPPRDWFPHIKDA...,chr10;390095;390274;-,390095:390274


In [11]:
oshu_df=ohsu_df[['gene_id','kmer','junction_coordinate']+EXPRESSION]

In [12]:
ETH_df.drop(columns=['strand'],inplace=True)
ETH_df.drop(columns=['coord'],inplace=True)
ETH_df.columns

Index(['gene_id', 'kmer', 'TCGA25131901A01R156513', 'TCGA25131301A01R156513',
       'TCGA61200801A02R156813', 'TCGA24143101A01R156613',
       'TCGA24229801A01R156913', 'junction_coordinate'],
      dtype='object')

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles

TEXT_SIZE = 65

def plot_venn(count1, count2, count_int, isDownload, title = 'Comparison of unique kmers counts between whole data sets',nameDownload='picture.png'):
    plt.figure(figsize=(50,50))
    total = count1 + count2 - count_int
    out = venn2(
        subsets = (count1 - count_int, count2 - count_int, count_int),
        set_labels = ('OVARIAN(ETH)', 'OVARIAN(OHSU)'),
        set_colors=[ETH_COLOR,OHSU_COLOR]
    )
    for text in out.set_labels:
        text.set_fontsize(TEXT_SIZE)
    for text in out.subset_labels:
        text.set_fontsize(TEXT_SIZE)
    plt.title(title, fontsize=TEXT_SIZE)
    #save_plot(title)
    if isDownload:
        plt.savefig(nameDownload, bbox_inches='tight')
    plt.show()
    return plt

def plot_venn_perc(count1, count2, count_int, isDownload, title = 'Comparison of unique kmers counts between whole data sets',nameDownload='picture.png'):
    plt.figure(figsize=(50,50))
    total = count1 + count2 - count_int
    out = venn2(
        subsets = (count1 - count_int, count2 - count_int, count_int),
        set_labels = ('OVARIAN(ETH)', 'OVARIAN(OHSU)'),
        set_colors=[ETH_COLOR,OHSU_COLOR],
        subset_label_formatter=lambda x: f"{(x/total):0.01%}"
    )
    for text in out.set_labels:
        text.set_fontsize(TEXT_SIZE)
    for text in out.subset_labels:
        text.set_fontsize(TEXT_SIZE)
    plt.title(title, fontsize=TEXT_SIZE)
    if isDownload:
        plt.savefig(nameDownload, bbox_inches='tight')
    plt.show()
    return plt

In [14]:
path_eth_df=create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_OVARIAN,NAME_TABLES,NAME_ETH_OVARIAN])
path_ohsu_df=create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_OVARIAN,NAME_TABLES,NAME_OHSU_OVARIAN])
ETH_df.to_csv(path_eth_df, header = True, sep='\t')
ohsu_df.to_csv(path_ohsu_df, header = True, sep='\t')

In [15]:
def common_genes(ohsu_df, eth_df, show_log = True):
    g1 = ohsu_df['gene_id'].unique()
    g2 = eth_df['gene_id'].unique()
    gene_names = list(set(g1) & set(g2))
    if show_log:
        print("OHSU number of genes:", len(g1))
        print("ETH number of genes:", len(g2))
        print("Number of common genes:", len(gene_names))
    return gene_names

def filter_df_common_kmers(df1, df2, show_log = True, col = 'kmer'):
    '''
    Filter input datframes, so that they have only records with common pairs gene + kmer
    '''
    if show_log:
        print('Initials sizes: ', len(df1), len(df2))
    gene_names = common_genes(df1, df2, show_log)
    df1 = df1[df1['gene_id'].isin(gene_names)]
    df1_grouped = df1.groupby('gene_id')
    df2 = df2[df2['gene_id'].isin(gene_names)]
    df2_grouped = df2.groupby('gene_id')
    dfs1 = []
    dfs2 = []
    for gene in tqdm.tqdm(gene_names):
        d1 = df1_grouped.get_group(gene)
        d2 = df2_grouped.get_group(gene)
        common_kmer = set(d1[col]) & set(d2[col])
        if len(common_kmer) == 0 and show_log:
            print("There are no common kmers for gene: ", gene)
        else:
            d1 = d1[d1[col].isin(common_kmer)]
            d2 = d2[d2[col].isin(common_kmer)]
            d1['gene_id'] = gene
            dfs1.append(d1)
            d2['gene_id'] = gene
            dfs2.append(d2)
    df1, df2 = pd.concat(dfs1, axis=0, ignore_index=True), pd.concat(dfs2, axis=0, ignore_index=True)
    if show_log:
        print('Result sizes: ', len(df1), len(df2))
    return df1, df2

def filter_df_common_coord(df1, df2):
    return filter_df_common_kmers(df1, df2, False, 'junction_coordinate')

In [16]:
oshu_df.columns

Index(['gene_id', 'kmer', 'junction_coordinate', 'TCGA25131901A01R156513',
       'TCGA25131301A01R156513', 'TCGA61200801A02R156813',
       'TCGA24143101A01R156613', 'TCGA24229801A01R156913'],
      dtype='object')

In [17]:
sample=[]
inter = []
kmer_inter=[]
eth_without_ohsu =  []
ohsu_withou_eth =  []
fs_ohsu =  []
kmer_ohsu=[]
fs_eth =  []
kmer_eth=[]
batch =  []
gene_id =  []
LANG='ENG'
OHSU_COLOR ='red' #'red'
ETH_COLOR = 'green'#'green'
OHSU_ETH_COLOR ='yellow' #'yellow'

if LANG == 'ENG':
    title_venn = 'Comparison of unique kmers ({exp_col})'
    title_venn_perc = 'Comparison of unique kmers in percentages ({exp_col})'
    title_barplot = 'Comparison of unique kmers ({exp_col})'
else:
    title_venn = 'Порівняння унікальних kmers ({exp_col})'
    title_venn_perc = 'Порівняння унікальних kmers у відсотках ({exp_col})'
    title_barplot = 'Порівняння унікальних kmers ({exp_col})'

out_table = {
    'sample':sample,
    'eth_without_ohsu':eth_without_ohsu,
    'kmer_eth':kmer_eth,
    'ohsu_without_eth':ohsu_withou_eth,
    'kmer_ohsu':kmer_ohsu,
    'inter':inter,
    'kmer_inter':kmer_inter,
    'full_size_ohsu': fs_ohsu,
    'full_size_eth': fs_eth,

    'coord_OHSU':[],
    'coord_ETH':[],
    'size_ohsu_coor' : [], 
    'size_eth_coor' : [], 
    'size_intersection_coor' : [], 
    'size_ohsu\eth_coor' : [], 
    'size_eth\ohsu_coor' : [],
    'eth_coor\inter_coor':[],
    'ohsu_coor\inter_coor':[],
    'eth_coor\ohsu_coor':[],
    'ohsu_coor\eth_coor':[],
    'inter_coor':[],
}
for exp_col in tqdm.tqdm(EXPRESSION):
    df1=ETH_df[ETH_df[exp_col] > 0]
    df2=oshu_df[oshu_df[exp_col] > 0]
    eth_kmers=set(df1['kmer'])
    ohsu_kmers=set(df2['kmer'])
    
    # df1, df2 = filter_df_common_kmers(eth_df[['gene_id', 'kmer', 'junction_coordinate']], ohsu_df[['gene_id', 'kmer', 'junction_coordinate']])
    out_table['coord_ETH'].append(df1['junction_coordinate'] if not df1.empty else 'None' )
    out_table['coord_OHSU'].append(df2['junction_coordinate'] if not df2.empty else 'None')
    ser1 = set(df1['junction_coordinate'])
    ser2 = set(df2['junction_coordinate'])
    out_table['size_ohsu_coor'].append(len(ser2))
    out_table['size_eth_coor'].append(len(ser1))
    out_table['size_ohsu\eth_coor'].append(len(df_ohsu_filter_coor:=ser2.difference(ser1)))
    out_table['size_eth\ohsu_coor'].append(len(df_eth_filter_coor:=ser1.difference(ser2)))
    out_table['size_intersection_coor'].append(len(df_inter_filter_coor:=ser2 & ser1))
    out_table['eth_coor\inter_coor'].append(df_eth_witout_inter_coor:=list(df_eth_filter_coor.difference(df_inter_filter_coor)))
    out_table['ohsu_coor\inter_coor'].append(df_ohsu_witout_inter_coor:=list(df_ohsu_filter_coor.difference(df_inter_filter_coor)))
    out_table['eth_coor\ohsu_coor'].append(df_eth_filter_coor)
    out_table['ohsu_coor\eth_coor'].append(df_ohsu_filter_coor)
    out_table['inter_coor'].append(list(df_inter_filter_coor))
    
    # intersect = set(ser1) & set(ser2)
    # s1, s2, s3 = len(ser1), len(ser2), len(intersect)
    # path_cn_coor=create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_OVARIAN,NAME_SAMPLES,exp_col,'coordinates','__count__','non filtered','coordinate.png'])
    # path_vn_coor=create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_OVARIAN,NAME_SAMPLES,exp_col,'coordinates','__perc__','non filtered','coordinate.png'])
    # path_cb_coor=create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_OVARIAN,NAME_SAMPLES,exp_col,'coordinates','__count__','non filtered','barplot_column_common_unique.png'])
    # draw_plot.plot_venn(s1, s2, s3, True,  title = exp_col,nameDownload= path_cn_coor)
    # draw_plot.plot_venn_perc(s1, s2, s3, True,  title = exp_col,nameDownload= path_vn_coor)
    # draw_plot.barplot_column_common_unique(s1, s2, s3, True, title = exp_col, ylabel = 'Number of junction coordinates',nameDownload= path_cb_coor)
    
    sample.append(exp_col)
    fs_ohsu.append(n_ohsu := len(ohsu_kmers))
    kmer_ohsu.append(list(ohsu_kmers))
    fs_eth.append(n_eth:=len(eth_kmers))
    kmer_eth.append(list(eth_kmers))
    eth_without_ohsu.append(len(eth_without_ohsu_kmers:=eth_kmers.difference(ohsu_kmers)))
    ohsu_withou_eth.append(len(oshu_without_ohsu_kmers:=ohsu_kmers.difference(eth_kmers)))
    inter.append(intersection:=len(intersection_kmers:= (eth_kmers & ohsu_kmers)))
    kmer_inter.append(list(intersection_kmers))
    # pwd = '/cluster/home/myurchikova/github/projects2020_ohsu/eth/learning_Master_thesis/'
    # directory_csv='DATA'
    # path_data =os.path.join(pwd,directory_csv)
    # if not os.path.exists(path_data):
    #     os.mkdir(path_data)
    # path_dir='OVARIAN'
    # path=os.path.join(path_data,path_dir)
    # if not os.path.exists(path):
    #     os.mkdir(path)
    # path_exp=os.path.join(path,exp_col)
    # if not os.path.exists(path_exp):
    #     os.mkdir(path_exp)
    # path_fc=os.path.join(path_exp,'__count__')
    # path_fv=os.path.join(path_exp,'__perc__')
    # if not os.path.exists(path_fc):
    #     os.mkdir(path_fc)
    # if not os.path.exists(path_fv):
    #     os.mkdir(path_fv)
    # path_cn=os.path.join(path_fc,'non-filtered.png')
    # path_vn=os.path.join(path_fv,'non-filtered.png')

    # path_cn=create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_OVARIAN,NAME_SAMPLES,exp_col,'kmers','__count__','non filtered','picture.png'])
    # path_vn=create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_OVARIAN,NAME_SAMPLES,exp_col,'kmers','__perc__','non filtered','picture.png'])

    # plot_venn(n_eth, n_ohsu, intersection, True, title = title_venn.format(exp_col = exp_col),nameDownload=path_cn)
    # plot_venn_perc(n_eth, n_ohsu, intersection, True, title = title_venn_perc.format(exp_col = exp_col),nameDownload=path_vn)
    
out_df_original=pd.DataFrame(data=out_table)

100%|█████████████████████████████████████████████| 5/5 [01:05<00:00, 13.15s/it]


In [18]:
# pwd = '/cluster/home/myurchikova/github/projects2020_ohsu/eth/learning_Master_thesis/'
# directory_csv='DATA/OVARIAN'
# path_data =os.path.join(pwd,directory_csv)
# name_non_filtering='out_df_non_filtering.csv'
path_non_filtering=create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_OVARIAN,NAME_TABLES,NAME_NON_FILTERING_OVARIAN])
# path_non_filtering=os.path.join(path_data,name_non_filtering)
out_df_original.to_csv(path_non_filtering, header = True, sep='\t')

In [19]:
out_df_original

,sample,eth_without_ohsu,kmer_eth,ohsu_without_eth,kmer_ohsu,inter,kmer_inter,full_size_ohsu,full_size_eth,coord_OHSU,...,size_ohsu_coor,size_eth_coor,size_intersection_coor,size_ohsu\eth_coor,size_eth\ohsu_coor,eth_coor\inter_coor,ohsu_coor\inter_coor,eth_coor\ohsu_coor,ohsu_coor\eth_coor,inter_coor
0,TCGA25131901A01R156513,816932,"[MPLKMKRTV, LSWWLCPMC, VKLVKKKEG, SFWGCRSSV, V...",428167,"[MPLKMKRTV, LSWWLCPMC, SEKRKKKKK, KDKKSGVYE, L...",379742,"[MPLKMKRTV, LSWWLCPMC, EAEIEYREN, KKGPEGRDL, E...",807909,1196674,0 48725:48803 0 ...,...,118453,134343,56375,62078,77968,"[46722340:46722809, 195882328:195883156, 67584...","[73518527:73518590, 126265476:126265736, 64066...","{46722340:46722809, 195882328:195883156, 67584...","{73518527:73518590, 126265476:126265736, 64066...","[31999144:31999305, 944719:945056, 29575377:29..."
1,TCGA25131301A01R156513,747604,"[MPLKMKRTV, VSPPQDFQR, GGVNVNVAN, MTGTAEMWT, C...",185089,"[MPLKMKRTV, SEKRKKKKK, IGIACVQAL, PQSRKYSSP, T...",304377,"[MPLKMKRTV, TRKMAEQER, QQFQNGVSW, PQSRKYSSP, L...",489466,1051981,0 48725:48803 0 ...,...,71516,115774,44970,26546,70804,"[46722340:46722809, 195882328:195883156, 67584...","[62692787:62694662, 94425764:94427031, 1236166...","{46722340:46722809, 195882328:195883156, 67584...","{62692787:62694662, 94425764:94427031, 1236166...","[57489345:57489427, 31999144:31999305, 3156629..."
2,TCGA61200801A02R156813,830107,"[LALLLSLEK, MPLKMKRTV, DSQQPPATV, VKLVKKKEG, V...",500417,"[LALLLSLEK, DSQQPPATV, MPLKMKRTV, KEKIIEGPD, Y...",384571,"[LALLLSLEK, DSQQPPATV, MPLKMKRTV, FDCPALSLT, P...",884988,1214678,0 48725:48803 0 ...,...,130035,136474,57147,72888,79327,"[46722340:46722809, 149370075:149372382, 50625...","[613572:614356, 110084678:110085743, 8863341:8...","{46722340:46722809, 50625464:50625578, 4481959...","{110084678:110085743, 613572:614356, 8863341:8...","[27440868:27441165, 57489345:57489427, 1797247..."
3,TCGA24143101A01R156613,733253,"[MPLKMKRTV, GYGHLGTWL, VSPPQDFQR, GGVNVNVAN, K...",218646,"[MPLKMKRTV, DVVDPRSAC, SEKRKKKKK, GYGHLGTWL, K...",282239,"[MPLKMKRTV, TRKMAEQER, GYGHLGTWL, QQFQNGVSW, K...",500885,1015492,48 820845:825763 48 ...,...,72839,110706,41714,31125,68992,"[46722340:46722809, 195882328:195883156, 67584...","[123616603:123619447, 44319793:44321527, 53213...","{46722340:46722809, 195882328:195883156, 67584...","{44319793:44321527, 123616603:123619447, 53213...","[31566295:31566809, 155190436:155191667, 15103..."
4,TCGA24229801A01R156913,876295,"[MPLKMKRTV, GGVNVNVAN, KEKIIEGPD, RIHTRSTNN, M...",505464,"[LALLLSLEK, MPLKMKRTV, SEKRKKKKK, IFNFNSNFN, K...",432843,"[LALLLSLEK, MPLKMKRTV, QVILKYRGL, KEKIIEGPD, Q...",938307,1309138,0 48725:48803 0 ...,...,139040,147851,64628,74412,83223,"[46722340:46722809, 50625464:50625578, 1265227...","[219174207:219174896, 1023854:1024514, 1236166...","{46722340:46722809, 50625464:50625578, 1265227...","{219174207:219174896, 1023854:1024514, 4501453...","[27440868:27441165, 57489345:57489427, 3199914..."
